# Model 3b - MLPRegressor with standardization & hyperparameter tuning (with 5-fold cross-validation)

In [1]:
# import packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats
import math
import os

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import RandomizedSearchCV, train_test_split, cross_validate
from sklearn.neural_network import MLPRegressor
from sklearn.pipeline import make_pipeline
from sklearn.metrics import mean_squared_error

from openpyxl import load_workbook

In [2]:
# get data
parentdir = os.path.dirname(os.getcwd())
data = pd.read_excel(parentdir+'\\Data\\SWedge Results.xlsx',sheet_name = "Probabilistic Values", engine='openpyxl')
data.head()

,Wedge ID,Safety Factor,Ln Safety Factor,Safety Factor Class,Safety Factor Class_1,Safety Factor Class_2,Ln Safety Factor Class_2,Wedge Volume (m3),Wedge Weight (MN),Plunge Line of Intersection (°),...,Water Pressure Joint 2 (MPa),Water Pressure Basal Joint (MPa),Water Pressure Tension Crack (MPa),Ponded Water Depth (m),Seismic Alpha,Seismic Plunge (°),Seismic Trend (°),Maximum Persistence Joint 1 (m),Maximum Persistence Joint 2 (m),Maximum Persistence Basal Joint (m)
0,BFA 60 [0],1.082239,0.079032,11.0,6.0,1.05,0.038481,6349.248707,171.429715,31.846178,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,NaN
1,BFA 60 [1],1.203906,0.185571,13.0,7.0,1.35,0.293893,7701.148241,207.931003,31.905513,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,NaN
2,BFA 60 [2],0.896601,-0.109144,9.0,5.0,0.75,-0.308093,2341.641868,63.224330,36.973415,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,NaN
3,BFA 60 [3],0.680996,-0.384199,7.0,4.0,0.75,-0.308093,155.345062,4.194317,54.969435,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,NaN
4,BFA 60 [4],1.263948,0.234240,13.0,7.0,1.35,0.293893,7468.340623,201.645197,29.688564,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,NaN


In [3]:
print(np.shape(data))

(5000, 92)


In [4]:
# get data specifically for the modelling (i.e., the inputs and output)
data_model = data[["Safety Factor","Dip of Joint 1 (°)","Dip Direction of Joint 1 (°)","Dip of Joint 2 (°)","Dip Direction of Joint 2 (°)","Dip of Slope (°)","Dip Direction of Slope (°)","Friction Angle of Joint 1 (°)","Friction Angle of Joint 2 (°)"]]
print(np.shape(data_model))
data_model.head()

(5000, 9)


,Safety Factor,Dip of Joint 1 (°),Dip Direction of Joint 1 (°),Dip of Joint 2 (°),Dip Direction of Joint 2 (°),Dip of Slope (°),Dip Direction of Slope (°),Friction Angle of Joint 1 (°),Friction Angle of Joint 2 (°)
0,1.082239,39.265808,120.865923,51.646228,221.979277,58.840543,182.626968,29.567773,29.522638
1,1.203906,38.981309,128.836961,57.766382,235.428421,63.804918,181.820235,32.713619,29.079492
2,0.896601,42.032968,117.504566,62.427355,217.726775,58.134485,180.398207,29.660213,27.455866
3,0.680996,69.264568,137.906910,66.183726,246.195109,61.968796,182.439496,30.866657,34.401616
4,1.263948,46.728166,121.226945,50.803809,241.060589,60.832522,179.091174,28.789453,28.613525


In [5]:
# remove any realizations that are not kinematically possible and any duplicates
data_model =  data_model.dropna()
data_model = data_model.drop_duplicates()
print(np.shape(data_model))

(4992, 9)


### Hyperparameter tuning

In [6]:
# hyperparameter grid for MLPRegressor
# number of neurons in hidden layer(s)
# note: 1 hidden layer - > (x,), 2 hidden layers -> (x,y), 3 hidden layers -> (x,y,z), etc
hidden_layer_sizes = [(10,),(15,),(20,),(10,10),(15,10),(20,10)]

# activation function for hidden layer
activation = ['tanh','relu']

# solver for weight optimization
solver = ['sgd','adam']

# alpha (strength of L2 regularization term)
# https://scikit-learn.org/stable/auto_examples/neural_networks/plot_mlp_alpha.html
alpha = [0.0001,0.001,0.01,0.1,1]

# initial learning rate
learning_rate_init = [0.001,0.01,0.1]

# maximum number of iterations
max_iter = [200,500]


# create the random grid
param_grid = {'mlpregressor__hidden_layer_sizes': hidden_layer_sizes,
              'mlpregressor__activation': activation,
              'mlpregressor__solver': solver,
              'mlpregressor__alpha': alpha,
              'mlpregressor__learning_rate_init': learning_rate_init,
              'mlpregressor__max_iter': max_iter}

print(param_grid)

{'mlpregressor__hidden_layer_sizes': [(10,), (15,), (20,), (10, 10), (15, 10), (20, 10)], 'mlpregressor__activation': ['tanh', 'relu'], 'mlpregressor__solver': ['sgd', 'adam'], 'mlpregressor__alpha': [0.0001, 0.001, 0.01, 0.1, 1], 'mlpregressor__learning_rate_init': [0.001, 0.01, 0.1], 'mlpregressor__max_iter': [200, 500]}


In [7]:
# random_state=123, early_stopping=True, validation_fraction=0.2

# function to get train & test r2 and RMSE for specified dataset size where hyperparameter tuning was performed
def hyperparam_results(data, dataset_size, param_grid):
    random_state_val = [0,1,42,123]
    param_grid = param_grid
    
    r2_train_subsample_list = []
    rmse_train_subsample_list = []
    mape_train_subsample_list = []
    r2_test_subsample_list = []
    rmse_test_subsample_list = []
    mape_test_subsample_list = []


    for x in range(0,4):
        # get subsample of data
        data_subsample = data_model.sample(n = dataset_size,random_state = 1)

        # train/test split with different random_state values (0, 1, 42, and 123)
        train_subsample, test_subsample = train_test_split(data_subsample, test_size=0.2, random_state=random_state_val[x])

        x_train_subsample = train_subsample[["Dip of Joint 1 (°)","Dip Direction of Joint 1 (°)","Dip of Joint 2 (°)","Dip Direction of Joint 2 (°)","Dip of Slope (°)","Dip Direction of Slope (°)","Friction Angle of Joint 1 (°)","Friction Angle of Joint 2 (°)"]]
        y_train_subsample = train_subsample[["Safety Factor"]]
        y_train_subsample = np.ravel(y_train_subsample)

        x_test_subsample = test_subsample[["Dip of Joint 1 (°)","Dip Direction of Joint 1 (°)","Dip of Joint 2 (°)","Dip Direction of Joint 2 (°)","Dip of Slope (°)","Dip Direction of Slope (°)","Friction Angle of Joint 1 (°)","Friction Angle of Joint 2 (°)"]]
        y_test_subsample = test_subsample[["Safety Factor"]]
        y_test_subsample = np.ravel(y_test_subsample)

        # hyperparameter tuning
        # make pipeline for MLPRegressor with pre-processing (standardizing the data)
        pipe_mlp = make_pipeline(StandardScaler(), MLPRegressor(random_state = 123,early_stopping=True,validation_fraction=0.2))
        random_search = RandomizedSearchCV(estimator=pipe_mlp, param_distributions=param_grid,n_iter=100, n_jobs=-1,cv=5,random_state=123)

        random_search.fit(x_train_subsample, y_train_subsample)

        ypred_mlp = random_search.predict(x_train_subsample)
        ypred_mlp = np.reshape(ypred_mlp,(len(ypred_mlp),1))

        # training r2 and rmse
        r2_train_subsample = random_search.score(x_train_subsample,y_train_subsample)
        rmse_train_subsample = math.sqrt(mean_squared_error(y_train_subsample,ypred_mlp))

        # append training r2 and rmse to their respective lists
        r2_train_subsample_list.append(r2_train_subsample)
        rmse_train_subsample_list.append(rmse_train_subsample)


        # test the mlp model
        # predict y test
        ypred_test_mlp = random_search.predict(x_test_subsample)
        ypred_test_mlp = np.reshape(ypred_test_mlp,(len(ypred_test_mlp),1))

        # test r2 and rmse
        r2_test_subsample = random_search.score(x_test_subsample,y_test_subsample)
        rmse_test_subsample = math.sqrt(mean_squared_error(y_test_subsample,ypred_test_mlp))

        # append test r2 and rmse to their respective lists
        r2_test_subsample_list.append(r2_test_subsample)
        rmse_test_subsample_list.append(rmse_test_subsample)
        
    return r2_train_subsample_list,rmse_train_subsample_list,r2_test_subsample_list,rmse_test_subsample_list
    

In [8]:
# MLP model results for dataset size = 100 data points
R2_train_100, rmse_train_100, R2_test_100, rmse_test_100 = hyperparam_results(data_model,100,param_grid)

In [9]:
# training and test r2 for MLP trained on 100 data points for four different random_state values in train/test split
print(R2_train_100)
print(R2_test_100)

[0.08880191378892821, 0.21278154032706964, 0.3788140215105845, 0.5206115440760606]
[0.4610277028308095, 0.336503760385949, 0.8878916610174372, 0.8108884929303347]


In [10]:
# training and test rmse for MLP trained on 100 data points for four different random_state values in train/test split
print(rmse_train_100)
print(rmse_test_100)

[1.2578695874090826, 1.1912412647157131, 1.0547549499571574, 0.932254448939627]
[0.4953782619824308, 0.3517017265836818, 0.16653704959200247, 0.17014629789372207]


In [13]:
# train - test r2 for MLP trained on 100 data points for four different random_state values in train/test split
R2_diff_100 = np.asarray(R2_train_100) - np.asarray(R2_test_100)
print(R2_diff_100)

# test - train rmse for MLP trained on 100 data points for four different random_state values in train/test split
rmse_diff_100 = np.asarray(rmse_test_100) - np.asarray(rmse_train_100)
print(rmse_diff_100)

[-0.37222579 -0.12372222 -0.50907764 -0.29027695]
[-0.76249133 -0.83953954 -0.8882179  -0.76210815]


In [14]:
# MLP model results for dataset size = 150 data points
R2_train_150, rmse_train_150, R2_test_150, rmse_test_150 = hyperparam_results(data_model,150,param_grid)

In [15]:
# training and test r2 for MLP trained on 150 data points for four different random_state values in train/test split
print(R2_train_150)
print(R2_test_150)

[0.5377828361885109, 0.858264506079799, 0.9146103560868711, 0.5428705971237759]
[0.8693453216302964, 0.017351872947760083, 0.37318327591327405, -1.144291718987422]


In [16]:
# training and test rmse for MLP trained on 150 data points for four different random_state values in train/test split
print(rmse_train_150)
print(rmse_test_150)

[0.7495254924504497, 0.4307083169548325, 0.14352757438532934, 0.7758513321510008]
[0.2587050146702111, 0.36928090039131917, 1.642216631665191, 0.440499398252651]


In [17]:
# train - test r2 for MLP trained on 150 data points for four different random_state values in train/test split
R2_diff_150 = np.asarray(R2_train_150) - np.asarray(R2_test_150)
print(R2_diff_150)

# test - train rmse for MLP trained on 150 data points for four different random_state values in train/test split
rmse_diff_150 = np.asarray(rmse_test_150) - np.asarray(rmse_train_150)
print(rmse_diff_150)

[-0.33156249  0.84091263  0.54142708  1.68716232]
[-0.49082048 -0.06142742  1.49868906 -0.33535193]


In [18]:
# MLP model results for dataset size = 200 data points
R2_train_200, rmse_train_200, R2_test_200, rmse_test_200 = hyperparam_results(data_model,200,param_grid)

In [19]:
# training and test r2 for MLP trained on 200 data points for four different random_state values in train/test split
print(R2_train_200)
print(R2_test_200)

[0.5610662938403107, 0.8542533123403682, 0.5656799772719978, 0.9894214501206057]
[0.2357732259128491, 0.4887406444960761, 0.8490924652686006, 0.8846778080267299]


In [20]:
# training and test rmse for MLP trained on 200 data points for four different random_state values in train/test split
print(rmse_train_200)
print(rmse_test_200)

[0.6768637729817131, 0.38445210483726827, 0.6637023500374829, 0.10526056817756295]
[0.30318085310710674, 0.348346770639324, 0.18687763298019194, 0.09445844567665601]


In [21]:
# train - test r2 for MLP trained on 200 data points for four different random_state values in train/test split
R2_diff_200 = np.asarray(R2_train_200) - np.asarray(R2_test_200)
print(R2_diff_200)

# test - train rmse for MLP trained on 200 data points for four different random_state values in train/test split
rmse_diff_200 = np.asarray(rmse_test_200) - np.asarray(rmse_train_200)
print(rmse_diff_200)

[ 0.32529307  0.36551267 -0.28341249  0.10474364]
[-0.37368292 -0.03610533 -0.47682472 -0.01080212]


In [22]:
# MLP model results for dataset size = 250 data points
R2_train_250, rmse_train_250, R2_test_250, rmse_test_250 = hyperparam_results(data_model,250,param_grid)

In [23]:
# training and test r2 for MLP trained on 250 data points for four different random_state values in train/test split
print(R2_train_250)
print(R2_test_250)

[0.9227686825125857, 0.5365414871107412, 0.45399371924840837, 0.9877708529986308]
[0.45104057762068106, 0.6007185401404103, 0.5121167492337805, 0.6861621349687117]


In [24]:
# training and test rmse for MLP trained on 250 data points for four different random_state values in train/test split
print(rmse_train_250)
print(rmse_test_250)

[0.12665168759716375, 0.6253237869176014, 0.6863015664155884, 0.1019052058821934]
[1.2182101533482979, 0.2892710403825874, 0.23764082047170867, 0.24348886627584143]


In [25]:
# train - test r2 for MLP trained on 250 data points for four different random_state values in train/test split
R2_diff_250 = np.asarray(R2_train_250) - np.asarray(R2_test_250)
print(R2_diff_250)

# test - train rmse for MLP trained on 250 data points for four different random_state values in train/test split
rmse_diff_250 = np.asarray(rmse_test_250) - np.asarray(rmse_train_250)
print(rmse_diff_250)

[ 0.4717281  -0.06417705 -0.05812303  0.30160872]
[ 1.09155847 -0.33605275 -0.44866075  0.14158366]


In [26]:
# MLP model results for dataset size = 750 data points
R2_train_750, rmse_train_750, R2_test_750, rmse_test_750 = hyperparam_results(data_model,750,param_grid)

In [27]:
# training and test r2 for MLP trained on 750 data points for four different random_state values in train/test split
print(R2_train_750)
print(R2_test_750)

[0.9484341405362441, 0.8225793604962152, 0.7655013975365361, 0.9936981885032529]
[0.5608251274943824, 0.8860252015950263, 0.9875852261784479, 0.6721198130503282]


In [28]:
# training and test rmse for MLP trained on 750 data points for four different random_state values in train/test split
print(rmse_train_750)
print(rmse_test_750)

[0.10754552867796552, 0.2752773833411017, 0.3088124044551475, 0.03624086670069489]
[0.6639628285456566, 0.14805862493255323, 0.058246557480081056, 0.5915301963653623]


In [29]:
# train - test r2 for MLP trained on 750 data points for four different random_state values in train/test split
R2_diff_750 = np.asarray(R2_train_750) - np.asarray(R2_test_750)
print(R2_diff_750)

# test - train rmse for MLP trained on 750 data points for four different random_state values in train/test split
rmse_diff_750 = np.asarray(rmse_test_750) - np.asarray(rmse_train_750)
print(rmse_diff_750)

[ 0.38760901 -0.06344584 -0.22208383  0.32157838]
[ 0.5564173  -0.12721876 -0.25056585  0.55528933]


In [30]:
# MLP model results for dataset size = 200 data points
R2_train_2000, rmse_train_2000, R2_test_2000, rmse_test_2000 = hyperparam_results(data_model,2000,param_grid)

In [31]:
# training and test r2 for MLP trained on 2000 data points for four different random_state values in train/test split
print(R2_train_2000)
print(R2_test_2000)

[0.910865945281249, 0.9435426456174045, 0.9382160514998862, 0.993370371335103]
[0.959021557483769, 0.9287069888345919, 0.8939300052698967, 0.8442070577197893]


In [32]:
# training and test r2 for MLP trained on 2000 data points for four different random_state values in train/test split
print(rmse_train_2000)
print(rmse_test_2000)

[0.16121762933486083, 0.13188980297082653, 0.13942003646241952, 0.039093591955569114]
[0.11085155628294135, 0.1291676998823314, 0.148530384664726, 0.2912193850503976]


In [33]:
# train - test r2 for MLP trained on 2000 data points for four different random_state values in train/test split
R2_diff_2000 = np.asarray(R2_train_2000) - np.asarray(R2_test_2000)
print(R2_diff_2000)

# test - train rmse for MLP trained on 2000 data points for four different random_state values in train/test split
rmse_diff_2000 = np.asarray(rmse_test_2000) - np.asarray(rmse_train_2000)
print(rmse_diff_2000)

[-0.04815561  0.01483566  0.04428605  0.14916331]
[-0.05036607 -0.0027221   0.00911035  0.25212579]
